<a href="https://colab.research.google.com/github/sugunayaparala/multiplatform-recomendation/blob/main/course_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U scikit-learn
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
udemy_data = pd.read_csv("/content/Udemy.csv")
coursera_data = pd.read_csv("/content/Coursera.csv")

In [3]:
udemy_data.rename(columns={'title': 'title', 'level': 'difficulty'}, inplace=True)
coursera_data.rename(columns={'course': 'title', 'level': 'difficulty'}, inplace=True)

In [4]:
udemy_data['platform'] = 'udemy'
coursera_data['platform'] = 'coursera'
combined_data = pd.concat([udemy_data, coursera_data], ignore_index=True)

In [5]:
combined_data['title'] = combined_data['title'].fillna('')
combined_data['description'] = combined_data['description'].fillna('')
combined_data['course_text'] = combined_data['title'] + " " + combined_data['description'] + " " + combined_data['platform']

In [6]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(combined_data['course_text'])

In [ ]:
course_similarity = cosine_similarity(tfidf_matrix)

In [ ]:
def recommend_similar_courses(course_title, top_n=5):
    course_index = combined_data[combined_data['title'] == course_title].index
    if len(course_index) == 0:
        return f"No course found with title: {course_title}"

    course_index = course_index[0]
    similar_courses = list(enumerate(course_similarity[course_index]))
    similar_courses = sorted(similar_courses, key=lambda x: x[1], reverse=True)
    similar_courses = similar_courses[1:top_n + 1]

    recommendations = [
        (combined_data.iloc[x[0]]['title'], combined_data.iloc[x[0]]['platform'], x[1])
        for x in similar_courses
    ]
    return recommendations

In [ ]:
def generate_sequential_recommendations(sequence_of_titles, top_n=5):
    sequential_recommendations = []

    for title in sequence_of_titles:
        recommendations = recommend_similar_courses(title, top_n)
        sequential_recommendations.append((title, recommendations))

    return sequential_recommendations

In [ ]:
sequence_of_titles = ["Python for Beginners", "Data Science Fundamentals"]
sequential_recommendations = generate_sequential_recommendations(sequence_of_titles)

for course_title, recommendations in sequential_recommendations:
    print(f"Sequential Recommendations for '{course_title}':")
    for recommended_title, platform, similarity_score in recommendations:
        print(f"  - {recommended_title} (Platform: {platform}, Similarity Score: {similarity_score:.4f})")
    print()